In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
import nltk
import re
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from nltk.tokenize import word_tokenize
from sklearn.tree import DecisionTreeClassifier

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\himanshu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\himanshu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\himanshu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
data = pd.read_csv('gender-classifier-DFE-791531.csv',encoding='latin1')

In [3]:
data.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,gender,gender:confidence,profile_yn,profile_yn:confidence,created,...,profileimage,retweet_count,sidebar_color,text,tweet_coord,tweet_count,tweet_created,tweet_id,tweet_location,user_timezone
0,815719226,False,finalized,3,10/26/15 23:24,male,1.0000,yes,1.0,12/5/13 1:48,...,https://pbs.twimg.com/profile_images/414342229...,0,FFFFFF,Robbie E Responds To Critics After Win Against...,NaN,110964,10/26/15 12:40,6.587300e+17,main; @Kan1shk3,Chennai
1,815719227,False,finalized,3,10/26/15 23:30,male,1.0000,yes,1.0,10/1/12 13:51,...,https://pbs.twimg.com/profile_images/539604221...,0,C0DEED,ÛÏIt felt like they were my friends and I was...,NaN,7471,10/26/15 12:40,6.587300e+17,NaN,Eastern Time (US & Canada)
2,815719228,False,finalized,3,10/26/15 23:33,male,0.6625,yes,1.0,11/28/14 11:30,...,https://pbs.twimg.com/profile_images/657330418...,1,C0DEED,i absolutely adore when louis starts the songs...,NaN,5617,10/26/15 12:40,6.587300e+17,clcncl,Belgrade
3,815719229,False,finalized,3,10/26/15 23:10,male,1.0000,yes,1.0,6/11/09 22:39,...,https://pbs.twimg.com/profile_images/259703936...,0,C0DEED,Hi @JordanSpieth - Looking at the url - do you...,NaN,1693,10/26/15 12:40,6.587300e+17,"Palo Alto, CA",Pacific Time (US & Canada)
4,815719230,False,finalized,3,10/27/15 1:15,female,1.0000,yes,1.0,4/16/14 13:23,...,https://pbs.twimg.com/profile_images/564094871...,0,0,Watching Neighbours on Sky+ catching up with t...,NaN,31462,10/26/15 12:40,6.587300e+17,NaN,NaN


In [4]:
data.shape

(20050, 26)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20050 entries, 0 to 20049
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   _unit_id               20050 non-null  int64  
 1   _golden                20050 non-null  bool   
 2   _unit_state            20050 non-null  object 
 3   _trusted_judgments     20050 non-null  int64  
 4   _last_judgment_at      20000 non-null  object 
 5   gender                 19953 non-null  object 
 6   gender:confidence      20024 non-null  float64
 7   profile_yn             20050 non-null  object 
 8   profile_yn:confidence  20050 non-null  float64
 9   created                20050 non-null  object 
 10  description            16306 non-null  object 
 11  fav_number             20050 non-null  int64  
 12  gender_gold            50 non-null     object 
 13  link_color             20050 non-null  object 
 14  name                   20050 non-null  object 
 15  pr

In [6]:
data.columns

Index(['_unit_id', '_golden', '_unit_state', '_trusted_judgments',
       '_last_judgment_at', 'gender', 'gender:confidence', 'profile_yn',
       'profile_yn:confidence', 'created', 'description', 'fav_number',
       'gender_gold', 'link_color', 'name', 'profile_yn_gold', 'profileimage',
       'retweet_count', 'sidebar_color', 'text', 'tweet_coord', 'tweet_count',
       'tweet_created', 'tweet_id', 'tweet_location', 'user_timezone'],
      dtype='object')

In [7]:
data.drop(['_unit_id','_last_judgment_at','created','fav_number','profileimage','retweet_count','tweet_coord',
         '_trusted_judgments', 'tweet_count', 'tweet_created', 'tweet_id', 'tweet_location', 'user_timezone', 
         '_golden','_unit_state', 'gender_gold', 'link_color', 'name', 'profile_yn_gold', 'sidebar_color', 
         'profile_yn', 'profile_yn:confidence','gender:confidence'], axis=1, inplace=True)

In [8]:
data.head()

,gender,description,text
0,male,i sing my own rhythm.,Robbie E Responds To Critics After Win Against...
1,male,I'm the author of novels filled with family dr...,ÛÏIt felt like they were my friends and I was...
2,male,louis whining and squealing and all,i absolutely adore when louis starts the songs...
3,male,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",Hi @JordanSpieth - Looking at the url - do you...
4,female,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...,Watching Neighbours on Sky+ catching up with t...


In [9]:
data.isnull().sum()

gender           97
description    3744
text              0
dtype: int64

In [10]:
data.dropna(inplace=True)

In [11]:
data

,gender,description,text
0,male,i sing my own rhythm.,Robbie E Responds To Critics After Win Against...
1,male,I'm the author of novels filled with family dr...,ÛÏIt felt like they were my friends and I was...
2,male,louis whining and squealing and all,i absolutely adore when louis starts the songs...
3,male,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",Hi @JordanSpieth - Looking at the url - do you...
4,female,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...,Watching Neighbours on Sky+ catching up with t...
...,...,...,...
20045,female,(rp),"@lookupondeath ...Fine, and I'll drink tea too..."
20046,male,"Whatever you like, it's not a problem at all. ...",Greg Hardy you a good player and all but don't...
20047,male,#TeamBarcelona ..You look lost so you should f...,You can miss people and still never want to se...
20048,female,Anti-statist; I homeschool my kids. Aspiring t...,@bitemyapp i had noticed your tendency to pee ...


In [12]:
data['gender'].value_counts()

female     5725
male       5469
brand      4328
unknown     702
Name: gender, dtype: int64

In [13]:
data=data[(data['gender']=='female')|(data['gender']=='male')]

In [14]:
data

,gender,description,text
0,male,i sing my own rhythm.,Robbie E Responds To Critics After Win Against...
1,male,I'm the author of novels filled with family dr...,ÛÏIt felt like they were my friends and I was...
2,male,louis whining and squealing and all,i absolutely adore when louis starts the songs...
3,male,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",Hi @JordanSpieth - Looking at the url - do you...
4,female,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...,Watching Neighbours on Sky+ catching up with t...
...,...,...,...
20045,female,(rp),"@lookupondeath ...Fine, and I'll drink tea too..."
20046,male,"Whatever you like, it's not a problem at all. ...",Greg Hardy you a good player and all but don't...
20047,male,#TeamBarcelona ..You look lost so you should f...,You can miss people and still never want to se...
20048,female,Anti-statist; I homeschool my kids. Aspiring t...,@bitemyapp i had noticed your tendency to pee ...


In [15]:
data['gender'].value_counts()

female    5725
male      5469
Name: gender, dtype: int64

In [16]:
for gen in data['gender']:
    if gen == 'male':
        data['gender'].replace({'male':1}, inplace = True)
    elif gen == 'female':
        data['gender'].replace({'female':0}, inplace = True)

data['gender'].value_counts()

0    5725
1    5469
Name: gender, dtype: int64

In [17]:
data

,gender,description,text
0,1,i sing my own rhythm.,Robbie E Responds To Critics After Win Against...
1,1,I'm the author of novels filled with family dr...,ÛÏIt felt like they were my friends and I was...
2,1,louis whining and squealing and all,i absolutely adore when louis starts the songs...
3,1,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",Hi @JordanSpieth - Looking at the url - do you...
4,0,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...,Watching Neighbours on Sky+ catching up with t...
...,...,...,...
20045,0,(rp),"@lookupondeath ...Fine, and I'll drink tea too..."
20046,1,"Whatever you like, it's not a problem at all. ...",Greg Hardy you a good player and all but don't...
20047,1,#TeamBarcelona ..You look lost so you should f...,You can miss people and still never want to se...
20048,0,Anti-statist; I homeschool my kids. Aspiring t...,@bitemyapp i had noticed your tendency to pee ...


In [18]:
data.iloc[:50,1]

0                                 i sing my own rhythm.
1     I'm the author of novels filled with family dr...
2                   louis whining and squealing and all
3     Mobile guy.  49ers, Shazam, Google, Kleiner Pe...
4     Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...
5                                    you don't know me.
7       The secret of getting ahead is getting started.
8            Pll Fan // Crazy about MCD // Ramen is bae
9     Renaissance art historian, University of Notti...
12                           Senior '16 . XI-XII-MMXIV.
14    im just here for tÌüp, bo burnham, and disney ...
16                                           JMKM_Òü
17    Over enthusiastic F1 fan.  Model collector, mu...
20    Artisan specializing in paper mache, print-mak...
21            He bled and died to take away my sins ÛÊ
22                                         union j xxxx
23                            You had me from the start
24                        BSc economics graduate

In [19]:
def clean(review):

    descrip = re.sub('[^a-zA-Z]', ' ', review)

    review = review.lower()
    return review

data['descrip_cleaned'] = pd.DataFrame(data['description'].apply(lambda x: clean(x)))

data.head()

,gender,description,text,descrip_cleaned
0,1,i sing my own rhythm.,Robbie E Responds To Critics After Win Against...,i sing my own rhythm.
1,1,I'm the author of novels filled with family dr...,ÛÏIt felt like they were my friends and I was...,i'm the author of novels filled with family dr...
2,1,louis whining and squealing and all,i absolutely adore when louis starts the songs...,louis whining and squealing and all
3,1,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",Hi @JordanSpieth - Looking at the url - do you...,"mobile guy. 49ers, shazam, google, kleiner pe..."
4,0,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...,Watching Neighbours on Sky+ catching up with t...,ricky wilson the best frontman/kaiser chiefs t...


In [20]:
data.head()

,gender,description,text,descrip_cleaned
0,1,i sing my own rhythm.,Robbie E Responds To Critics After Win Against...,i sing my own rhythm.
1,1,I'm the author of novels filled with family dr...,ÛÏIt felt like they were my friends and I was...,i'm the author of novels filled with family dr...
2,1,louis whining and squealing and all,i absolutely adore when louis starts the songs...,louis whining and squealing and all
3,1,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",Hi @JordanSpieth - Looking at the url - do you...,"mobile guy. 49ers, shazam, google, kleiner pe..."
4,0,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...,Watching Neighbours on Sky+ catching up with t...,ricky wilson the best frontman/kaiser chiefs t...


In [21]:
data['descrip_cleaned'].replace('[@+]', "", regex=True,inplace=True)
data['descrip_cleaned'].replace('[()]', "", regex=True,inplace=True)
data['descrip_cleaned'].replace('[#+]', "", regex=True,inplace=True)
url_regex = '''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))'''
data['descrip_cleaned'] = data['descrip_cleaned'].replace(url_regex, "", regex=True)

In [22]:
data

,gender,description,text,descrip_cleaned
0,1,i sing my own rhythm.,Robbie E Responds To Critics After Win Against...,i sing my own rhythm.
1,1,I'm the author of novels filled with family dr...,ÛÏIt felt like they were my friends and I was...,i'm the author of novels filled with family dr...
2,1,louis whining and squealing and all,i absolutely adore when louis starts the songs...,louis whining and squealing and all
3,1,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",Hi @JordanSpieth - Looking at the url - do you...,"mobile guy. 49ers, shazam, google, kleiner pe..."
4,0,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...,Watching Neighbours on Sky+ catching up with t...,ricky wilson the best frontman/kaiser chiefs t...
...,...,...,...,...
20045,0,(rp),"@lookupondeath ...Fine, and I'll drink tea too...",rp
20046,1,"Whatever you like, it's not a problem at all. ...",Greg Hardy you a good player and all but don't...,"whatever you like, it's not a problem at all. ..."
20047,1,#TeamBarcelona ..You look lost so you should f...,You can miss people and still never want to se...,teambarcelona ..you look lost so you should fo...
20048,0,Anti-statist; I homeschool my kids. Aspiring t...,@bitemyapp i had noticed your tendency to pee ...,anti-statist; i homeschool my kids. aspiring t...


In [23]:
data['descrip_cleaned'] = [nltk.word_tokenize(tweet) for tweet in data['descrip_cleaned']]
descrip_new = []
for each_row in data['descrip_cleaned']:
    descrip_new.append([i for i in each_row if i.isalpha()])
descrip_new_alpha = []

In [24]:
stop_words = set(stopwords.words('english'))
# now remove them from the list
for each_row in descrip_new:
    descrip_new_alpha.append([i for i in each_row if i not in stop_words])

In [25]:
description_new_lemma = []
lemma = nltk.WordNetLemmatizer()
for each_row in descrip_new_alpha:
    description_new_lemma.append([lemma.lemmatize(word) for word in each_row])
#put back the new sentences
data['descrip_cleaned'] = description_new_lemma
data['descrip_cleaned'] = [" ".join(desc) for desc in data['descrip_cleaned'].values]

In [26]:
data['text_cleaned'] = pd.DataFrame(data['text'].apply(lambda y: clean(y)))
data.head()

,gender,description,text,descrip_cleaned,text_cleaned
0,1,i sing my own rhythm.,Robbie E Responds To Critics After Win Against...,sing rhythm,robbie e responds to critics after win against...
1,1,I'm the author of novels filled with family dr...,ÛÏIt felt like they were my friends and I was...,author novel filled family drama romance,ûïit felt like they were my friends and i was...
2,1,louis whining and squealing and all,i absolutely adore when louis starts the songs...,louis whining squealing,i absolutely adore when louis starts the songs...
3,1,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",Hi @JordanSpieth - Looking at the url - do you...,mobile guy shazam google kleiner perkins yahoo...,hi @jordanspieth - looking at the url - do you...
4,0,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...,Watching Neighbours on Sky+ catching up with t...,ricky wilson best chief best band xxxx thank k...,watching neighbours on sky+ catching up with t...


In [27]:
data['text_cleaned'].replace('[@+]', "", regex=True,inplace=True)
data['text_cleaned'].replace('[()]', "", regex=True,inplace=True)
data['text_cleaned'].replace('[#+]', "", regex=True,inplace=True)
url_regex = '''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))'''
data['text_cleaned'] = data['text_cleaned'].replace(url_regex, "", regex=True)

In [28]:
data['text_cleaned'] = [nltk.word_tokenize(tweet) for tweet in data['text_cleaned']]
text_new = []
for each_row in data['text_cleaned']:
    text_new.append([i for i in each_row if i.isalpha()])
text_new_alpha = []


In [29]:
for each_row in text_new:
    text_new_alpha.append([i for i in each_row if i not in stop_words])


In [30]:
text_new_lemma = []
for each_row in text_new_alpha:
    text_new_lemma.append([lemma.lemmatize(word) for word in each_row])
#put back the new sentences
data['text_cleaned'] = text_new_lemma
data['text_cleaned'] = [" ".join(desc) for desc in data['text_cleaned'].values]


In [31]:
data.head()


,gender,description,text,descrip_cleaned,text_cleaned
0,1,i sing my own rhythm.,Robbie E Responds To Critics After Win Against...,sing rhythm,robbie e responds critic win eddie edward worl...
1,1,I'm the author of novels filled with family dr...,ÛÏIt felt like they were my friends and I was...,author novel filled family drama romance,felt like friend living story http retired http
2,1,louis whining and squealing and all,i absolutely adore when louis starts the songs...,louis whining squealing,absolutely adore louis start song hit hard fee...
3,1,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",Hi @JordanSpieth - Looking at the url - do you...,mobile guy shazam google kleiner perkins yahoo...,hi jordanspieth looking url use ifttt typicall...
4,0,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...,Watching Neighbours on Sky+ catching up with t...,ricky wilson best chief best band xxxx thank k...,watching neighbour sky catching neighbs xxx xxx


In [32]:
data.drop(['description', 'text'], axis = 1, inplace = True)
data.head()

,gender,descrip_cleaned,text_cleaned
0,1,sing rhythm,robbie e responds critic win eddie edward worl...
1,1,author novel filled family drama romance,felt like friend living story http retired http
2,1,louis whining squealing,absolutely adore louis start song hit hard fee...
3,1,mobile guy shazam google kleiner perkins yahoo...,hi jordanspieth looking url use ifttt typicall...
4,0,ricky wilson best chief best band xxxx thank k...,watching neighbour sky catching neighbs xxx xxx


In [33]:
cv = CountVectorizer(max_features = 1000)
x = cv.fit_transform(data['descrip_cleaned']).toarray()
x1 = cv.fit_transform(data['text_cleaned']).toarray()

In [34]:
A = pd.DataFrame(x)
B = pd.DataFrame(x1)

In [35]:
A

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11189,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11190,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11191,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11192,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
B

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11189,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11190,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11191,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11192,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
X = pd.concat([B, A], join = 'outer', axis = 1)
X.shape

(11194, 2000)

In [38]:
X

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11189,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11190,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11191,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11192,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
data['gender'].shape


(11194,)

In [40]:
x = np.array(X)
y = np.array(data['gender'])

In [41]:
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state = 0)

# GaussianNB

In [42]:
gnb_model = GaussianNB()
gnb_model.fit(X_train , y_train)
y_pred = gnb_model.predict(X_test)

In [43]:
accuracy = accuracy_score(y_test, y_pred)
print("accuracy: %.2f%%" % (accuracy*100.0))


accuracy: 62.89%


In [44]:
confusion_matrix(y_test, y_pred)


array([[785, 347],
       [484, 623]], dtype=int64)

In [45]:
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.62      0.69      0.65      1132
           1       0.64      0.56      0.60      1107

    accuracy                           0.63      2239
   macro avg       0.63      0.63      0.63      2239
weighted avg       0.63      0.63      0.63      2239



# we are using grid search for hyperparameter optimizing the hyperparameters of the models

In [46]:
param_grid_nb = {'var_smoothing': np.logspace(0, -9, num = 100)}


In [47]:
nb_model_grid = GridSearchCV(estimator = gnb_model, param_grid = param_grid_nb, verbose = 1, cv = 3, n_jobs = -1)


In [48]:
nb_model_grid.fit(X_train, y_train)


Fitting 3 folds for each of 100 candidates, totalling 300 fits


GridSearchCV(cv=3, estimator=GaussianNB(), n_jobs=-1,
             param_grid={'var_smoothing': array([1.00000000e+00, 8.11130831e-01, 6.57933225e-01, 5.33669923e-01,
       4.32876128e-01, 3.51119173e-01, 2.84803587e-01, 2.31012970e-01,
       1.87381742e-01, 1.51991108e-01, 1.23284674e-01, 1.00000000e-01,
       8.11130831e-02, 6.57933225e-02, 5.33669923e-02, 4.32876128e-02,
       3.51119173e-02, 2.8480358...
       1.23284674e-07, 1.00000000e-07, 8.11130831e-08, 6.57933225e-08,
       5.33669923e-08, 4.32876128e-08, 3.51119173e-08, 2.84803587e-08,
       2.31012970e-08, 1.87381742e-08, 1.51991108e-08, 1.23284674e-08,
       1.00000000e-08, 8.11130831e-09, 6.57933225e-09, 5.33669923e-09,
       4.32876128e-09, 3.51119173e-09, 2.84803587e-09, 2.31012970e-09,
       1.87381742e-09, 1.51991108e-09, 1.23284674e-09, 1.00000000e-09])},
             verbose=1)

In [49]:
nb_model_grid.best_params_


{'var_smoothing': 0.006579332246575682}

In [50]:
y_pred_hyper = nb_model_grid.predict(X_test)


In [51]:
print(confusion_matrix(y_test, y_pred_hyper))


[[881 251]
 [552 555]]


In [52]:
accuracy = accuracy_score(y_test, y_pred_hyper)
print("accuracy after tuning: %.2f%%" % (accuracy*100.0))


accuracy after tuning: 64.14%


In [53]:
print(classification_report(y_test, y_pred_hyper))


              precision    recall  f1-score   support

           0       0.61      0.78      0.69      1132
           1       0.69      0.50      0.58      1107

    accuracy                           0.64      2239
   macro avg       0.65      0.64      0.63      2239
weighted avg       0.65      0.64      0.63      2239



# XGBClassifier

In [54]:
xgb_model = XGBClassifier(max_depth = 5, min_child_weight = 1)
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [55]:
y_pred2 = xgb_model.predict(X_test)


In [56]:
accuracy = accuracy_score(y_test, y_pred2)
print('Accuracy: %.2f%%' % (accuracy * 100.0))


Accuracy: 63.56%


In [57]:
confusion_matrix(y_test, y_pred2)


array([[867, 265],
       [551, 556]], dtype=int64)

In [58]:
print(classification_report(y_test, y_pred2))


              precision    recall  f1-score   support

           0       0.61      0.77      0.68      1132
           1       0.68      0.50      0.58      1107

    accuracy                           0.64      2239
   macro avg       0.64      0.63      0.63      2239
weighted avg       0.64      0.64      0.63      2239



# Hyperparameter optimization

In [59]:
xgb_param_grid = {
              "max_depth": [3, 5],
              "min_child_weight": [1, 2],
              }

In [60]:
xgb_grid = GridSearchCV(estimator = xgb_model, param_grid = xgb_param_grid, verbose = 1, cv = 3, n_jobs = -1)


In [61]:
xgb_grid.fit(X_train, y_train)


Fitting 3 folds for each of 4 candidates, totalling 12 fits


GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_bin=None,
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=5,
                                     max_le

In [62]:
xgb_grid.best_params_


{'max_depth': 5, 'min_child_weight': 2}

In [63]:
y_pred2_hyper = xgb_grid.predict(X_test)


In [64]:
accuracy = accuracy_score(y_test, y_pred2_hyper)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 63.64%


In [65]:
confusion_matrix(y_test, y_pred2_hyper)


array([[870, 262],
       [552, 555]], dtype=int64)

In [66]:
print(classification_report(y_test, y_pred2_hyper))


              precision    recall  f1-score   support

           0       0.61      0.77      0.68      1132
           1       0.68      0.50      0.58      1107

    accuracy                           0.64      2239
   macro avg       0.65      0.63      0.63      2239
weighted avg       0.65      0.64      0.63      2239



# RandomForestClassifier

In [67]:
rfc=RandomForestClassifier()


In [68]:
rfc.fit(X_train,y_train)

RandomForestClassifier()

In [69]:
y_pred=rfc.predict(X_test)


In [70]:
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: %.2f%%' % (accuracy * 100.0))


Accuracy: 64.45%


In [71]:
confusion_matrix(y_test, y_pred)


array([[738, 394],
       [402, 705]], dtype=int64)

In [72]:
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.65      0.65      0.65      1132
           1       0.64      0.64      0.64      1107

    accuracy                           0.64      2239
   macro avg       0.64      0.64      0.64      2239
weighted avg       0.64      0.64      0.64      2239



# KNeighborsClassifier

In [73]:
knc=KNeighborsClassifier()


In [74]:
knc.fit(X_train,y_train)

KNeighborsClassifier()

In [75]:
y_pred=knc.predict(X_test)


In [76]:
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: %.2f%%' % (accuracy * 100.0))


Accuracy: 51.14%


In [77]:
confusion_matrix(y_test, y_pred)


array([[684, 448],
       [646, 461]], dtype=int64)

In [78]:
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.51      0.60      0.56      1132
           1       0.51      0.42      0.46      1107

    accuracy                           0.51      2239
   macro avg       0.51      0.51      0.51      2239
weighted avg       0.51      0.51      0.51      2239



# DecisionTreeClassifier

In [81]:
dtc=DecisionTreeClassifier()


In [82]:
dtc.fit(X_train,y_train)

DecisionTreeClassifier()

In [83]:
y_pred=dtc.predict(X_test)


In [84]:
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: %.2f%%' % (accuracy * 100.0))


Accuracy: 58.87%


In [85]:
confusion_matrix(y_test, y_pred)


array([[653, 479],
       [442, 665]], dtype=int64)

In [86]:
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.60      0.58      0.59      1132
           1       0.58      0.60      0.59      1107

    accuracy                           0.59      2239
   macro avg       0.59      0.59      0.59      2239
weighted avg       0.59      0.59      0.59      2239

